# Python with APIs
Exchanges via HTTP using `requests` package

In [106]:
import requests

## Example

### Agify API

Example route for Agify API

In [107]:
agify_dan = "https://api.agify.io/?name=dan"

Extract response

In [108]:
# Run a GET request
agify_answer = requests.get(agify_dan)
agify_answer_txt = agify_answer.text
agify_answer_json = agify_answer.json()

print(f"Answer status_code: {agify_answer}")

print(type(agify_answer_txt))
print(type(agify_answer_json))

print(agify_answer_txt)
print(agify_answer_json)

Answer status_code: <Response [200]>
<class 'str'>
<class 'dict'>
{"count":67782,"name":"dan","age":66}
{'count': 67782, 'name': 'dan', 'age': 66}


## Introductory Exercises

In [109]:
# Exercise 1
# Write a script that asks the user for their first name
# and responds with a personalized message
# using the agify API

# https://api.agify.io/?name=YOUR_NAME
# Example response: {"name":"YOUR_NAME","age":30,"count":12345}

def get_data(name,api):
    response = requests.get(f"{api},{name}")
    if response.status_code == 200:
        return response.json()
    else:
        return None

user_name = input("Please enter your first name: ")
data = get_data(user_name,"https://api.agify.io/?name=")

if data:
    estimated_age = data['age']
    count = data['count']
    print("Your name is ",user_name,", your age is ",estimated_age)
else:
    print("Fail")


Please enter your first name: guillaume
Your name is  guillaume , your age is  43


In [110]:
# Exercise 2
# Write a script that asks the user for their first name
# and responds with a personalized message
# using the genderize API
# https://api.genderize.io/?name=YOUR_NAME


user_name = input("Please enter your first name: ")
data = get_data(user_name,"https://api.genderize.io/?name=")

if data:
    estimated_gender = data['gender']
    proba = data['probability'] * 100
    print(f"With the name {user_name}, the probability of being {estimated_gender} is {proba}%")
else:
    print("Fail")


Please enter your first name: guillaume
With the name guillaume, the probability of being male is 100.0%


In [111]:
# Exercise 3
# Write a script that asks the user for their first name
# and responds with a personalized message
# using the nationalize API
# https://api.nationalize.io/?name=YOUR_NAME

user_name = input("Please enter your first name: ")
data = get_data(user_name, "https://api.nationalize.io/?name=")

if data and 'country' in data:
    print(f"Based on your name, {user_name}, here are your top estimated nationalities and their probabilities:")
    for country in data['country']:
        country_id = country['country_id']
        proba = country['probability'] * 100
        print(f"{country_id}: {proba:.2f}%")
else:
    print("Failed to fetch estimated nationality data.")

Please enter your first name: guillaume
Based on your name, guillaume, here are your top estimated nationalities and their probabilities:
FR: 21.99%
BE: 17.03%
HT: 9.91%
CI: 7.24%
CH: 4.89%


In [112]:
# Exercise 3.1
# Parsing the response from the nationalize API
# Get the most probable country and its percentage
# Example response: {"name":"YOUR_NAME","country":[{"country_id":"FR","probability":0.75},{"country_id":"BE","probability":0.25}]}
# Hint: use the max() function with a lambda function
# https://docs.python.org/3/library/functions.html#max
# https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions

def get_data(name,api):
    response = requests.get(f"{api},{name}")
    if response.status_code == 200:
        return response.json()
    else:
        return None

user_name = input("Please enter your first name: ")
data = get_data(user_name, "https://api.nationalize.io/?name=")

if data and 'country' in data and data['country']:
    most_probable = max(data['country'], key=lambda country: country['probability'])
    country_id = most_probable['country_id']
    proba = most_probable['probability'] * 100
    print(f"Based on your name, {user_name}, the most probable nationality is {country_id} with a probability of {proba:.2f}%.")
else:
    print("Failed to fetch estimated nationality data or no data available.")



Please enter your first name: guillaume
Based on your name, guillaume, the most probable nationality is FR with a probability of 21.99%.


In [113]:
# Exercise 4
# Use BoredAPI : https://www.boredapi.com/
# Documentation : https://www.boredapi.com/documentation

# 1. Write a script that generates random activities
response = requests.get("https://www.boredapi.com/api/activity")
if response.status_code == 200:
  if response.json():
      print(f"Q1. Activity: {response.json()['activity']}")
  else:
      print("Q1. Failed to fetch activity.")


# 3. Write a script that generates random activities for 4 participants
response = requests.get(f"https://www.boredapi.com/api/activity?participants=4")
if response.status_code == 200:
  if response.json():
      print(f"Q3. Activity for 4 participants: {response.json()['activity']}")
  else:
      print("Q3. Failed to fetch activity.")


# 4. Write a script that generates random activities for 4 participants and of type "recreational"

response = requests.get(f"https://www.boredapi.com/api/activity?participants=4&type=recreational")
if response.status_code == 200:
  if response.json():
      print(f"Q4. Recreational activity for 4 participants: {response.json()['activity']}")
  else:
      print("Q4. Failed to fetch activity.")


# 5. Write a script that generates random activities for 2 participants and that does not require equipment
response = requests.get(f"https://www.boredapi.com/api/activity?participants=2")
if response.status_code == 200:
  if response.json():
      print(f"Q5. Activity for 2 participants: {response.json()['activity']}")
  else:
      print("Q5. Failed to fetch activity.")



Q1. Activity: Visit a nearby museum
Q3. Activity for 4 participants: Take a spontaneous road trip with some friends
Q4. Recreational activity for 4 participants: Go see a Broadway production
Q5. Activity for 2 participants: Go swimming with a friend


## Intermediate exercises

In [114]:
# OpenDomesday
# https://opendomesday.org/api/

# Exercise 1
# Write a script that displays all the counties
# using the OpenDomesday API.

response = requests.get("https://opendomesday.org/api/1.0/county/")
if response.status_code == 200:
  if response.json():
      print("Counties from OpenDomesday:")
      for county in response.json():
          print(county['name'])
  else :
      print("Failed to fetch counties.")

Counties from OpenDomesday:
Kent
Sussex
Surrey
Hampshire
Berkshire
Wiltshire
Dorset
Somerset
Devon
Cornwall
Middlesex
Hertfordshire
Buckinghamshire
Gloucestershire
Oxfordshire
Worcestershire
Herefordshire
Cambridgeshire
Huntingdonshire
Bedfordshire
Northamptonshire
Leicestershire
Warwickshire
Staffordshire
Shropshire
Cheshire
Derbyshire
Nottinghamshire
Rutland
Yorkshire
Lincolnshire
Claims: YB
Claims: YC
Claims: LC
Claims: HC
Claims: YS
Essex
Norfolk
Suffolk
Lancashire


In [115]:
# Exercise 2
# Write a script that displays the information
# of the county "Derbyshire".

if response.status_code == 200:
    counties = response.json()
    for county in counties:
        if "Derbyshire".lower() == county.get('name', '').lower():
            if county:
                print(county)
            else:
                print(f"Failed to fetch information.")

{'id': 'dby', 'name': 'Derbyshire', 'name_slug': 'derbyshire', 'places_in_county': [{'id': 1036}, {'id': 2558}, {'id': 3016}, {'id': 4791}, {'id': 6093}, {'id': 8701}, {'id': 8951}, {'id': 9101}, {'id': 11441}, {'id': 10771}, {'id': 16116}, {'id': 20861}, {'id': 22251}, {'id': 22571}, {'id': 22611}, {'id': 24741}, {'id': 25536}, {'id': 19061}, {'id': 30246}, {'id': 31896}, {'id': 32521}, {'id': 32981}, {'id': 33916}, {'id': 41346}, {'id': 41788}, {'id': 41801}, {'id': 45821}, {'id': 47401}, {'id': 47411}, {'id': 52361}, {'id': 52596}, {'id': 53901}, {'id': 54446}, {'id': 54646}, {'id': 55736}, {'id': 56786}, {'id': 57061}, {'id': 60236}, {'id': 60351}, {'id': 60816}, {'id': 63606}, {'id': 65368}, {'id': 73221}, {'id': 73731}, {'id': 73741}, {'id': 91}, {'id': 2623}, {'id': 3011}, {'id': 3941}, {'id': 4046}, {'id': 5016}, {'id': 5676}, {'id': 7111}, {'id': 7116}, {'id': 7451}, {'id': 9056}, {'id': 10981}, {'id': 11656}, {'id': 11941}, {'id': 12751}, {'id': 13401}, {'id': 14081}, {'id': 

In [116]:
# Exercise 3
# Now that we have the ids for all the places in Derbyshire, we can load all their details...
# And from their details, we can list all the details of their manors.
# Go fetch the data!
# P.S.: remember to save the data to avoid downloading it every time

import json

def save_details(name_county):
    url1 = "https://opendomesday.org/api/1.0/county"
    url2 = "https://opendomesday.org/api/1.0/place"
    params = {'name': name_county}

    try:

        reponse_comte = requests.get(url1, params=params)
        reponse_comte.raise_for_status()
        data_comte = reponse_comte.json()


        for county in data_comte:
            if county['name'].lower() == name_county.lower():
                details_lieux = []


                for lieu in county['places_in_county']:
                    reponse_lieu = requests.get(f"{url2}/{lieu['id']}")
                    reponse_lieu.raise_for_status()
                    details_lieux.append(reponse_lieu.json())


                with open(f'manors_{name_county}.json', 'w', encoding='utf-8') as fichier:
                    json.dump(details_lieux, fichier, ensure_ascii=False, indent=4)

                print(f"Details of {name_county} have been saved.")
                break

    except requests.RequestException as e:
        print(f"Something wrong happened : {e}")

save_details("Derbyshire")

Details of Derbyshire have been saved.


In [117]:
# Exercise 4
# Now that we have a quantity of raw data, we will extract the interesting parts.
# In our case, we want to count the money paid by each manor and compare it to the number of ploughs it has.
# - Can you find the corresponding json fields?
# - Then, you can list these numbers for each manor in Derbyshire.
# - And format this in an appropriate comma-separated values (CSV) file.

import csv

csv_file = 'manors_Derbyshire.json'

with open(csv_file, 'r', encoding='utf-8') as fichier:
    lieux = json.load(fichier)
    manor_ids = [manoir['id'] for lieu in lieux for manoir in lieu.get('manors', [])]

filename = "manors_data.csv"

with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Manor ID", "Monetary Value in 1086", "Total Ploughs"])

    for manor_id in manor_ids:
        url = f"https://opendomesday.org/api/1.0/manor/{manor_id}/"
        response = requests.get(url)
        if response.status_code == 200:
            manor_data = response.json()
            monetary_value = manor_data.get("value86", "Unknown")
            total_ploughs = manor_data.get("totalploughs", "Unknown")
            writer.writerow([manor_id, monetary_value, total_ploughs])
        else:
            print(f"Failed to fetch data for manor ID {manor_id}")

print(f"Details fetched to {filename}.")

Details fetched to manors_data.csv.


In [118]:
# Exercise 5
# What is the richest manor in Derbyshire?

filename = "manors_data.csv"
value = 0
manor = None

with open(filename, mode='r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            val = float(row["Monetary Value in 1086"])
            if val > value:
                value = val
                manor = row["Manor ID"]
        except ValueError:
            continue

print(f"The richest manor of Derbyshire is {manor} (value of {value})")

The richest manor of Derbyshire is 12958 (value of 40.0)


In [119]:
# Exercise 6
# Give the total value paid by Derbyshire.

filename = "manors_data.csv"
total_value = 0

with open(filename, mode='r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            value86 = float(row["Monetary Value in 1086"])
            total_value += value86
        except ValueError:

            continue

print(f"The total value paid by Derbyshire is {total_value}.")


The total value paid by Derbyshire is 1098.6182999999996.


In [125]:
# Exercise 7
# Create a Python class.
# It must include all the previous functionalities.
# Refactor your code to make it readable, efficient, and maintainable.

class OpenDomesdayHelper:
    def __init__(self):
        self.base_url = "https://opendomesday.org/api/1.0/"

    def fetch_counties(self):
        response = requests.get(f"{self.base_url}county/")
        if response.status_code == 200 and response.json():
            print("Counties from OpenDomesday:")
            for county in response.json():
                print(county['name'])
        else:
            print("Failed to fetch counties.")

    def fetch_county_info(self, county_name):
        response = requests.get(f"{self.base_url}county/")
        if response.status_code == 200:
            counties = response.json()
            for county in counties:
                if county_name.lower() == county.get('name', '').lower():
                    print(county)
                    return county
            print(f"County {county_name} not found.")
        else:
            print("Failed to fetch counties.")

    def save_place_details(self, county_name):
        places_in_county = self.fetch_county_info(county_name).get('places_in_county', [])
        place_details = []

        for place in places_in_county:
            response = requests.get(f"{self.base_url}place/{place['id']}")
            if response.status_code == 200:
                place_details.append(response.json())

        with open(f'manors_{county_name}.json', 'w', encoding='utf-8') as file:
            json.dump(place_details, file, ensure_ascii=False, indent=4)
        print(f"Details of {county_name} have been saved.")

    def generate_csv_from_json(self, json_file, csv_file):
        with open(json_file, 'r', encoding='utf-8') as file:
            places = json.load(file)
            manor_ids = [manor['id'] for place in places for manor in place.get('manors', [])]

        with open(csv_file, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Manor ID", "Monetary Value in 1086", "Total Ploughs"])

            for manor_id in manor_ids:
                response = requests.get(f"{self.base_url}manor/{manor_id}/")
                if response.status_code == 200:
                    manor_data = response.json()
                    monetary_value = manor_data.get("value86", "Unknown")
                    total_ploughs = manor_data.get("totalploughs", "Unknown")
                    writer.writerow([manor_id, monetary_value, total_ploughs])
                else:
                    print(f"Failed to fetch data for manor ID {manor_id}")

        print(f"Details fetched to {csv_file}.")


    def find_richest_manor(self, csv_file):
        richest_value = 0
        richest_manor = None
        with open(csv_file, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                try:
                    val = float(row["Monetary Value in 1086"])
                    if val > richest_value:
                        richest_value = val
                        richest_manor = row["Manor ID"]
                except ValueError:
                    continue
        print(f"The richest manor in Derbyshire is {richest_manor} with a value of {richest_value}.")

    def calculate_total_value(self, csv_file):
        total_value = 0
        with open(csv_file, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                try:
                    value86 = float(row["Monetary Value in 1086"])
                    total_value += value86
                except ValueError:
                    continue
        print(f"The total value paid by Derbyshire is {total_value}.")


Ex7 = OpenDomesdayHelper()
Ex7.fetch_counties()
Ex7.fetch_county_info("Derbyshire")
Ex7.save_place_details("Derbyshire")
Ex7.save_place_details("Derbyshire")
Ex7.generate_csv_from_json("manors_Derbyshire.json","manors_Derbyshire.csv")
Ex7.find_richest_manor("manors_Derbyshire.csv")
Ex7.calculate_total_value("manors_Derbyshire.csv")



Counties from OpenDomesday:
Kent
Sussex
Surrey
Hampshire
Berkshire
Wiltshire
Dorset
Somerset
Devon
Cornwall
Middlesex
Hertfordshire
Buckinghamshire
Gloucestershire
Oxfordshire
Worcestershire
Herefordshire
Cambridgeshire
Huntingdonshire
Bedfordshire
Northamptonshire
Leicestershire
Warwickshire
Staffordshire
Shropshire
Cheshire
Derbyshire
Nottinghamshire
Rutland
Yorkshire
Lincolnshire
Claims: YB
Claims: YC
Claims: LC
Claims: HC
Claims: YS
Essex
Norfolk
Suffolk
Lancashire
{'id': 'dby', 'name': 'Derbyshire', 'name_slug': 'derbyshire', 'places_in_county': [{'id': 1036}, {'id': 2558}, {'id': 3016}, {'id': 4791}, {'id': 6093}, {'id': 8701}, {'id': 8951}, {'id': 9101}, {'id': 11441}, {'id': 10771}, {'id': 16116}, {'id': 20861}, {'id': 22251}, {'id': 22571}, {'id': 22611}, {'id': 24741}, {'id': 25536}, {'id': 19061}, {'id': 30246}, {'id': 31896}, {'id': 32521}, {'id': 32981}, {'id': 33916}, {'id': 41346}, {'id': 41788}, {'id': 41801}, {'id': 45821}, {'id': 47401}, {'id': 47411}, {'id': 52361}, 

In [121]:
# Exercise 8 (optional)
# Add to your class a system for error handling.
# It must manage the following errors:
# - Connection error
# - Parsing error
# - Request error
# - Response error
# - Parameter error

#######################################################
################## YOUR ANSWER HERE ##################
#######################################################

